<a href="https://colab.research.google.com/github/kridtapon/Fisher-MACD-Momentum-Shift/blob/main/Fisher_MACD_Momentum_Shift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.6 MB/s eta 0:00:00


In [2]:
pip install --upgrade yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.52
    Uninstalling yfinance-0.2.52:
      Successfully uninstalled yfinance-0.2.52


In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from itertools import product

# Function to calculate MACD
def calculate_macd(df, short_window, long_window, signal_window):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

# Function to calculate Fisher Transform
def calculate_fisher_transform(df, period):
    high_rolling = df['High'].rolling(window=period).max()
    low_rolling = df['Low'].rolling(window=period).min()
    X = 2 * ((df['Close'] - low_rolling) / (high_rolling - low_rolling) - 0.5)
    fisher = 0.5 * np.log((1 + X) / (1 - X))
    return fisher

# Define the stock symbol and time period
symbol = 'ADP'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Define parameter ranges
short_windows = range(10, 21)
long_windows = range(21, 36)
signal_windows = range(10, 21)
fisher_periods = range(5, 30, 2)

# Store optimization results
results = []

# Grid search over parameter combinations
for short_w, long_w, signal_w, fisher_p in product(short_windows, long_windows, signal_windows, fisher_periods):
    df['MACD'], df['MACD_Signal'] = calculate_macd(df, short_w, long_w, signal_w)
    df['Fisher'] = calculate_fisher_transform(df, fisher_p)

    # Filter data for the test period (2020-2025)
    df_filter = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

    df_filter['Entry'] = (df_filter['MACD'] > 0) & (df_filter['MACD'].shift(1) <= 0) & (df_filter['Fisher'] > 0)
    df_filter['Exit'] = (df_filter['MACD'] < 0) & (df_filter['MACD'].shift(1) >= 0) & (df_filter['Fisher'] < 0)

    # Backtest
    portfolio = vbt.Portfolio.from_signals(
        close=df_filter['Close'],
        entries=df_filter['Entry'],
        exits=df_filter['Exit'],
        init_cash=100_000,
        fees=0.001
    )

    total_return = portfolio.total_return()
    results.append((short_w, long_w, signal_w, fisher_p, total_return))

# Find best parameters
best_params = max(results, key=lambda x: x[4])
print(f"Best Parameters: MACD Short={best_params[0]}, MACD Long={best_params[1]}, Signal={best_params[2]}, Fisher={best_params[3]}")
print(f"Best Total Return: {best_params[4]:.2f}%")

# Calculate MACD and Fisher Transform
df['MACD'], df['MACD_Signal'] = calculate_macd(df, best_params[0], best_params[1], best_params[2])
df['Fisher'] = calculate_fisher_transform(df, best_params[3])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Define Entry and Exit signals based on MACD Zero Line and Fisher Transform
df['Entry'] = (df['MACD'] > 0) & (df['MACD'].shift(1) <= 0) & (df['Fisher'] > 0)
df['Exit'] = (df['MACD'] < 0) & (df['MACD'].shift(1) >= 0) & (df['Fisher'] < 0)

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


Streaming output truncated to the last 5000 lines.
<ipython-input-6-09f06bea38ce>:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-09f06bea38ce>:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-09f06bea38ce>:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

Best Parameters: MACD Short=10, MACD Long=23, Signal=10, Fisher=29
Best Total Return: 1.21%
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           221017.269145
Total Return [%]                       121.017269
Benchmark Return [%]                    91.069632
Max Gross Exposure [%]                      100.0
Total Fees Paid                       6325.609129
Max Drawdown [%]                        20.699691
Max Drawdown Duration                       215.0
Total Trades                                   18
Total Closed Trades                            18
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            55.555556
Best Trade [%]                          22.103399
Worst Trade [%]                         -4.540731
Avg Winn

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate MACD and Signal line
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """
    Calculate MACD line and Signal line.
    """
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

# Function to calculate Fisher Transform
def calculate_fisher_transform(df, period=10):
    """
    Calculate Fisher Transform.
    """
    high_rolling = df['High'].rolling(window=period).max()
    low_rolling = df['Low'].rolling(window=period).min()

    # Calculate X (normalized price movement)
    X = 2 * ((df['Close'] - low_rolling) / (high_rolling - low_rolling) - 0.5)

    # Fisher Transform
    fisher = 0.5 * np.log((1 + X) / (1 - X))

    return fisher

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate MACD and Fisher Transform
df['MACD'], df['MACD_Signal'] = calculate_macd(df)
df['Fisher'] = calculate_fisher_transform(df, period=30)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Define Entry and Exit signals based on MACD Zero Line and Fisher Transform
df['Entry'] = (df['MACD'] > 0) & (df['MACD'].shift(1) <= 0) & (df['Fisher'] > 0)
df['Exit'] = (df['MACD'] < 0) & (df['MACD'].shift(1) >= 0) & (df['Fisher'] < 0)

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           244592.105215
Total Return [%]                       144.592105
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                       4517.058427
Max Drawdown [%]                        57.721987
Max Drawdown Duration                       435.0
Total Trades                                   16
Total Closed Trades                            15
Total Open Trades                               1
Open Trade PnL                       -3105.734576
Win Rate [%]                                 40.0
Best Trade [%]                         130.529927
Worst Trade [%]                        -37.272401
Avg Winning Trade [%]                   43.749079
Avg Losing Trade [%]                   -10.142136
